In [ ]:
from libpy import readfile

#filename = "2024/day24/input/sample.txt"
#filename = "2024/day24/input/sample2.txt"
filename = "2024/day24/input/input.txt"

lines = readfile.lines(filename)

inputs = {}
outputs = []
wires = {}

read_wires = False
for line in lines:
    if line == "":
        read_wires = True
    elif read_wires:
        p = line.split(" ")
        wires[p[4]] = (p[0], p[1], p[2])
        if p[4][0] == 'z':
            outputs.append(p[4])
    else:
        p = line.split(": ")
        inputs[p[0]] = p[1] == "1"
        
print(inputs)
print(wires)
print(outputs)

Part A

In [ ]:
calculations = {
    'AND': lambda a,b: a and b,
    'OR': lambda a,b: a or b,
    'XOR': lambda a,b: a is not b
}

def get_wire_value(output):
    if output in inputs:
        return inputs[output]
    
    formula = wires[output]
    input1 = get_wire_value(formula[0])
    input2 = get_wire_value(formula[2])
    
    return calculations[formula[1]](input1, input2)

result = ""
for output in sorted(outputs, reverse=True):
    result += "1" if get_wire_value(output) else "0"

print(int(result, 2))

Part B

In [ ]:
from collections import defaultdict


calculations = {
    'AND': lambda a,b: a and b,
    'OR': lambda a,b: a or b,
    'XOR': lambda a,b: a is not b
}

def get_all_inputs_for_output(wires, output):
    result = set()
    if output in inputs:
        return result
    result.add(output)
    
    formula = wires[output]
    result = set(list(result) + list(get_all_inputs_for_output(wires, formula[0])))
    result = set(list(result) + list(get_all_inputs_for_output(wires, formula[2])))
    
    return result

def get_wire_value(wires, output):
    if output in inputs:
        return inputs[output]
    
    formula = wires[output]
    input1 = get_wire_value(wires, formula[0])
    input2 = get_wire_value(wires, formula[2])
    
    return calculations[formula[1]](input1, input2)

new_wires = wires.copy()
#new_wires["z09"] = wires["cwp"]

x = ""
y = ""
for input in sorted(inputs, reverse=True):
    if input[0] == "x":
        x += "1" if get_wire_value(new_wires, input) else "0"
    else:
        y += "1" if get_wire_value(new_wires, input) else "0"

expected = bin((int(x, 2) + int(y, 2)))[2:]
result = ""
actual = {}
original_errors = 0
suspects = []
for output in sorted(outputs, reverse=True):
    r = get_wire_value(new_wires, output)
    actual[output] = r
    result += "1" if r else "0"
    if r != (expected[(len(expected) - int(output[1:]) - 1)] == "1"):
        print(output, r, sorted(list(get_all_inputs_for_output(new_wires, output))))
        suspects.append(get_all_inputs_for_output(new_wires, output))
        original_errors += 1

for s in suspects:
    for w in s:
        exists_in_other_suspect = False
        for s2 in suspects:
            if s != s2:
                if w in s2:
                    exists_in_other_suspect = True
        if not exists_in_other_suspect:
            print("Suspect:", w)

print(original_errors)

print(" " + x)
print(" " + y)
print("-" * (len(x)+1) + " ADD")
print(expected)

print("------")
print(result)

# suspects = list()
# for i in range(len(expected)):
#     if expected[i] != result[i]:
#         wire = "z"
#         if i < 10:
#             wire += "0"
#         wire += str(i)
#         suspects = suspects + list(get_all_inputs_for_output(wire))
#         print(i, "is", result[i], "instead of", expected[i])

# print(len(set(suspects)))
# print(313-92)
